## Transfer Learning with VGG-16 CNN + AUG LB 0.1712

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os

root_dir = Path('/content/drive/MyDrive/Colab Notebooks')
data_dir = root_dir / 'Iceberg'
predict_dir = root_dir 

os.listdir(data_dir)

['train.json', 'sample_submission.csv', 'test.json']

## Transfer Learning with VGG-16
## data augmentation

## Load data

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [4]:
train = pd.read_json(data_dir / 'train.json')
target_train = train['is_iceberg']
test = pd.read_json(data_dir / 'test.json')

In [5]:
target_train = train['is_iceberg']
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce')
train['inc_angle'] = train['inc_angle'].fillna(method='pad')
X_angle = train['inc_angle']
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle = test['inc_angle']

In [6]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)

In [7]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

In [8]:
from matplotlib import pyplot
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LeakyReLU, PReLU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input	

### Data Augmentation

In [9]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 64
gen = ImageDataGenerator(horizontal_flip=True,
                        vertical_flip=True, 
                        width_shift_range=0.,
                        height_shift_range=0.,
                        channel_shift_range=0.,
                        zoom_range=0.2,
                        rotation_range=10)

In [10]:
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=55)
    genX2 = gen.flow(X1, X2, batch_size=batch_size, seed=55)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield [X1i[0], X2i[1]], X1i[1]

In [11]:
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=10, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

In [12]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name='angle')
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                      input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    
    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(inputs=[base_model.input, input_2], outputs=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', 
                 optimizer=sgd, metrics=['accuracy'])
    return model

In [13]:
def myAngleCV(X_train, X_angle, X_test):
    K = 3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log = 0
    y_valid_pred_log = 0.0 * target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout = target_train[test_idx]
        
        X_angle_cv = X_angle[train_idx]
        X_angle_hold = X_angle[test_idx]
        
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        
        galaxyModel = getVggAngleModel()
        galaxyModel.fit_generator(
            gen_flow, 
            steps_per_epoch=24, 
            epochs=100,
            shuffle=True,
            verbose=1, 
            validation_data=([X_holdout, X_angle_hold], Y_holdout), 
            callbacks=callbacks
        )
        
        galaxyModel.load_weights(filepath=file_path)
        score = galaxyModel.evaluate([X_train_cv, X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        
        score = galaxyModel.evaluate([X_holdout, X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        
        pred_valid = galaxyModel.predict([X_holdout, X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])
        
        temp_test = galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log += temp_test.reshape(temp_test.shape[0])
        
        temp_train = galaxyModel.predict([X_train, X_angle])
        y_train_pred_log += temp_train.reshape(temp_train.shape[0])
    
    y_test_pred_log = y_test_pred_log / K
    y_train_pred_log = y_train_pred_log / K
    
    print('\n Train Log Loss Validation= ', log_loss(target_train, y_train_pred_log))
    print('\n Test Log Loss Validation= ', log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [14]:
preds = myAngleCV(X_train, X_angle, X_test)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/100
24/24 [==============================] - 24s 365ms/step - loss: 0.6430 - accuracy: 0.6486 - val_loss: 0.3551 - val_accuracy: 0.8280
Epoch 2/100
24/24 [==============================] - 6s 246ms/step - loss: 0.4030 - accuracy: 0.8167 - val_loss: 0.3207 - val_accuracy: 0.8449
Epoch 3/100
24/24 [==============================] - 7s 306ms/step - loss: 0.3133 - accuracy: 0.8525 - val_loss: 0.2560 - val_accuracy: 0.8822
Epoch 4/100
24/24 [==============================] - 6s 237ms/step - loss: 0.2949 - accuracy: 0.8701 - val_loss: 0.2542 - val_accuracy: 0.8785
Epoch 5/100
24/24 [==============================] - 6s 234ms/step - loss: 0.2624 - accuracy: 0.8832 - val_loss: 0.2299 - val_accuracy: 0.9103
Epoch 6/100
24/24 [==============================] - 6s 237ms/step - loss: 0.2601 - accuracy: 0.8906 - val_loss: 0.2236 - val_accuracy: 0.9028
Epoch 7/100
24/24 [==============================] - 5s 225ms/step - loss: 0.2284 - accuracy: 0.9084 - val_loss: 0.2374 - val_accuracy: 0.895

In [15]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['is_iceberg'] = preds
submission.to_csv('vgg.csv', index=False)

### test score: 0.17870